This code in written with the help of PyTorch Image Segmentation Demo:

https://learnopencv.com/pytorch-for-beginners-semantic-segmentation-using-torchvision/

# Steps

1. reading CSV file


In [ ]:
import pandas as pd
train_csv_path = '../input/siim-isic-melanoma-classification/train.csv'
jpeg_dir = '../input/siim-isic-melanoma-classification/jpeg/train'
train_df = pd.read_csv(train_csv_path)
train_df.head()

2. Reading Image

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

row = train_df.iloc[5]
print(row)
img = Image.open(f"{jpeg_dir}/{row[0]}.jpg")
plt.imshow(img); plt.show()

3. Load NN and declare transformations

In [ ]:
# Models and transformations
import torchvision.transforms as T
from torchvision import models
import torch
import torch.nn as nn

trf = T.Compose([T.Resize(512),
                 T.CenterCrop(512),
                 T.ToTensor(), 
                 T.Normalize(mean = [0.485, 0.456, 0.406], 
                             std = [0.229, 0.224, 0.225])])

class NullNet(nn.Module): 
    def __init__(self):
        super(NullNet, self).__init__()
    def forward(self, x):
        return x
    

inp = trf(img).unsqueeze(0)    
model_ft = models.resnet18(pretrained=True)
model_ft.fc = NullNet()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

4. Apply NN to single Image and observe

In [ ]:
# Pass the input through the net
inp = inp.to(device)
out = model_ft(inp)

print ("Input Shape:",inp.shape)
print ("Feature Shape:",out.shape)
print ("Length of Train Data:", len(train_df))

5. Saving Features to a CSV file

In [ ]:
import csv
import time
import numpy as np

since = time.time()
iter1=0
file = open('train_features.csv', 'a+', newline ='')
# writing the data into the file 
with file:     
    write = csv.writer(file) 
    
    for row in train_df.iloc:
        img = Image.open(f"{jpeg_dir}/{row[0]}.jpg")
        inp = trf(img).unsqueeze(0)
        inp = inp.to(device)
        out = model_ft(inp)

        output_format = np.concatenate((  row ,(out[0,:].cpu().detach().numpy() )),axis=0)
        write.writerows([output_format]) 

        iter1 = iter1 + 1

        if iter1 % 2000 == 1999:
            time_elapsed = time.time() - since
            print('Time from start {:.0f}m {:.0f}s'.format(
            time_elapsed // 60, time_elapsed % 60))
            print('Percentage complete: {:4f}'.format(100*iter1/(len(train_df))))
            #break # delete this break to save features from the entire dataset